In [1]:
import sys



from utilmy import pd_read_file
from rag.engine_kg import TripletExtractorModel, dbsql_fetch_text_by_doc_ids
# from rag.bench import glob_glob_last_modified
tpe = TripletExtractorModel()



Model loaded in 2.42 seconds


# Debug triplet generation

In [2]:
filepath = "ztmp/bench/ag_news/kg_triplets/*/*.parquet"
df = pd_read_file(filepath)

In [3]:
x = dbsql_fetch_text_by_doc_ids(text_ids=[10002150938768847559, 10002920186622135836])
def get_text_length_byid(text_id):
    try:
        record,*_ = dbsql_fetch_text_by_doc_ids(text_ids=[text_id])
        length= len(record["text"].split())
        return length
    except Exception as err:
        pass
    return 0
print(get_text_length_byid(10002150938768847559))

29


In [4]:
df.shape

(77036, 5)

In [5]:
dbsql_fetch_text_by_doc_ids(text_ids=[13168321631958298019])

[{'text_id': '13168321631958298019',
  'text': "Best Asian Tourism Destinations The new APMF survey of the best Asian tourism destinations has just kicked off, but it's crowded at the top, with Chiang Mai in Thailand just leading from perennial favourites Hong Kong, Bangkok and Phuket in Thailand, and Bali in  Indonesia. Be one of the first to vote and let us know your reasons."}]

In [6]:
df["type"].value_counts()

type
country                                             9841
member of sports team                               5965
instance of                                         3101
position held                                       2477
located in the administrative territorial entity    2395
                                                    ... 
constellation                                          1
lyrics by                                              1
country <triplet>\Darfur                               1
located on street circuit                              1
area                                                   1
Name: count, Length: 224, dtype: int64

In [7]:
# count group by text_id

df[["text_id", "head"]].groupby("text_id").count().reset_index()["head"].value_counts()

head
1     31942
2     12349
3      2402
4      1601
5       555
6       321
7       132
8        67
9        24
10       16
12       12
11        6
13        3
Name: count, dtype: int64

In [8]:
new_df = df[["text_id", "head"]].groupby("text_id").count().reset_index()

In [9]:
new_df[new_df["head"]==1] 

,text_id,head
5,10002150938768847559,1
7,10002920186622135836,1
8,10003267026028842906,1
10,10004552131248127602,1
11,1000472594856410479,1
...,...,...
49423,9997977433422292775,1
49426,9998550306966083495,1
49427,9998595610247887485,1
49428,9998607669409074929,1


In [10]:
new_df["word_count"] = new_df["text_id"].apply(get_text_length_byid)

In [11]:
new_df[new_df["head"]==1]

,text_id,head,word_count
5,10002150938768847559,1,29
7,10002920186622135836,1,41
8,10003267026028842906,1,53
10,10004552131248127602,1,31
11,1000472594856410479,1,44
...,...,...,...
49423,9997977433422292775,1,51
49426,9998550306966083495,1,51
49427,9998595610247887485,1,25
49428,9998607669409074929,1,34


In [12]:
# sort by word_count
new_df.sort_values("word_count", ascending=False)

,text_id,head,word_count
15090,15033374514112618740,1,168
13475,1448207204043896405,2,147
40327,6918921984522097614,2,146
5473,11820452355826007925,2,145
28630,3056806053299193426,4,145
...,...,...,...
45590,8711276450989850140,2,11
31753,4066509443622662641,2,10
45085,8536315245805628285,2,10
29640,3382653792506686967,1,10


In [13]:
text_id = 9997977433422292775
record, *_ = dbsql_fetch_text_by_doc_ids(text_ids=[text_id])
print([record["text"]])
triplets = tpe.extract([record["text"]])
triplets

['Oil prices hover above US\\$43 in Asia but market calm after Saudi &lt;b&gt;...&lt;/b&gt; SINGAPORE : Oil prices pushed past 43 dollars in Asian trading but the market was quite calm as the focus shifted to OPEC following a bloody attack on the US consulate in the Saudi port city of Jeddah.']


[[{'head': 'bloody attack on the US consulate',
   'type': 'location',
   'tail': 'Jeddah'}]]

**Findings:**  
1. Number of triplets not dependent on length but number of entities present in document.  
2. Some times same information is being indexed in 2 triplets  
    Eg:
    ```
        [[{'head': 'NASA', 'type': 'subsidiary', 'tail': 'Ames Research Center'},
      {'head': 'Ames Research Center',
       'type': 'parent organization',
       'tail': 'NASA'}]]
    ```
    Effect on scoring: This might lead to skewness in scoring.
3. 

# Debug failed queries

In [14]:
import os
import glob


# get failed neo4j questions

def glob_glob_last_modified(dirpath):
    # print(dirpath)
    files = sorted(glob.glob(dirpath), key=os.path.getctime, reverse=True)
    if len(files) > 0:
        return files[0]  # Latest file


dfmetrics_file = glob_glob_last_modified("ztmp/bench/ag_news/neo4j/*/*/dfmetrics.csv")
neo_df = pd_read_file(dfmetrics_file, sep="\t")


In [15]:
neo_df = neo_df[neo_df["istop_k"]==0]
neo_df["text"] = neo_df["id"].apply(lambda x: dbsql_fetch_text_by_doc_ids(text_ids=[x])[0]["text"])
neo_df=neo_df[["id", "text"]]

In [16]:
from rag.relation_extraction import extract_relations
def add_triplets_to_df(df):
    df["triplets"] = df["text"].apply(lambda x: extract_relations(x))
    df = neo_df.explode(column="triplets")
    df["head"] = df["triplets"].apply(lambda x: x["head"])
    df["type"] = df["triplets"].apply(lambda x: x["type"])
    df["tail"] = df["triplets"].apply(lambda x: x["tail"])
    df = df[["id","text","head","type", "tail"]]
    return df

In [17]:
result_df = add_triplets_to_df(neo_df)
result_df.to_csv("ztmp/spacy_triplets.csv", index=False)

relation_matches [have halted, halted, showed, said, said on]
relation_matches [soar, soar to, posing, toppling, straining, present]
relation_matches [End, End Up, ended, ended slightly, stayed, stayed near, surged, surged past, offsetting]
relation_matches [Fell, Fell in, fell, fell by, said]
relation_matches [Soaring, are expected, expected, are expected to, expected to, hang, hang over]
relation_matches [Pump, can do, do, douse, scorching, oversupplied, oversupplied by, said, warning, could fall, fall, could fall sharply, fall sharply]
relation_matches [Should Up, Up, should consider, consider, increasing, said, said on]
relation_matches [got, got off, got off to, sidestepped]
relation_matches [tumbled, tumbled broadly, tumbled broadly on, showing, cast, draw, fund, growing]
relation_matches [could be bracing, be bracing, bracing]
relation_matches [keeps, growing, show, continues, continues to, grow, are, may slow, slow, may slow down, slow down]
relation_matches [begins, begins on,